In [1]:
import pandas as pd
# from sklearn.feature_extraction.text import CountVectorizer

full = pd.read_csv("../../data/poptrag_lyrics_genres_corpus_filtered_english_lemmatized.csv")
full.info()

# top 20 most common words in the lyrics
# def print_most_common_words(corpus, lyrics_column, top_n=20):
#     vectorizer = CountVectorizer(
#                 ngram_range=(1, 1),
#                 token_pattern=r"\b[\w']+\b",
#                 lowercase=True,
#             )
#     matrix = vectorizer.fit_transform(full[lyrics_column])
#     sum_words = matrix.sum(axis=0)
#     words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
#     words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
#     print(words_freq[:20])

# print_most_common_words(full, "full_lyrics")
# print("=" * 60)
# print_most_common_words(full, "lyrics_lemmatized")

<class 'pandas.DataFrame'>
RangeIndex: 111938 entries, 0 to 111937
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                111938 non-null  int64  
 1   track.s.id                111938 non-null  str    
 2   track.s.title             111937 non-null  str    
 3   track.s.firstartist.name  111938 non-null  str    
 4   album.s.title             111938 non-null  str    
 5   album.s.releaseyear       111938 non-null  int64  
 6   track.s.popularity        111938 non-null  int64  
 7   track.language            111938 non-null  str    
 8   full_lyrics               111938 non-null  str    
 9   cat5                      111938 non-null  str    
 10  pmax5                     111938 non-null  float64
 11  nmax5                     111938 non-null  float64
 12  cat12                     111938 non-null  str    
 13  pmax12                    111938 non-null  float64
 14 

In [2]:
from helpers.LyricsClassficationExperiment import LyricsClassificationExperiment

exp_fs = LyricsClassificationExperiment(
    corpus=full, 
    genrecol="cat12",
    lyricscol="full_lyrics", 
    artistcol="track.s.firstartist.name",
    yearcol="album.s.releaseyear", 
    output_dir="cat5_mock_experiment_fs",
    test_size=0.2,
    random_state=42, 
    subsample_debug=0.005,
)
exp_fs.compute_fs_ngram_features(min_artists=20, top_n_per_genre_and_ngram=100)
fs_features = exp_fs.X_train.keys()
print(exp_fs)
print(type(exp_fs.X_train))
# exp_fs.train_fixed_parametrer_logistic_regression()
# exp_fs.show_model_evaluation()
# exp_fs.show_top_coefficients_per_genre()

N:\Materialien\Promotion\LyricsGenreRecognition\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
N:\Materialien\Promotion\LyricsGenreRecognition\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Extracted unigrams:
  - Unique: 6,022
  - Shape: (448, 6022)
  - Examples: ['tense', 'cinnamon', 'anymore', 'gospel', 'foes']
Extracted bigrams:
  - Unique: 31,519
  - Shape: (448, 31519)
  - Examples: ['rode impalas', 'blood on', 'addict just', 'tell your', 'free through']
Extracted trigrams:
  - Unique: 48,978
  - Shape: (448, 48978)
  - Examples: ['town well i', 'carry you but', 'all among the', 'your hand your', 'i was ready']
Total unique n-grams: 389
LyricsClassificationExperiment with 11 genres
Train size: 448 samples
Test size: 111 samples
# of features: 389
Feature type: Fell-Spohrleder (2014) N-grams (top 100 (per genre and ngram type), min. 20 artists)
Model not yet trained.
Output directory: cat5_mock_experiment_fs

<class 'pandas.DataFrame'>


In [ ]:
import numpy as np
PARAM_SPACE = {
    "C": [np.log10(0.001), np.log10(100.0)],
    "l1_ratio": [0.0, 1.0],
}

# exp_fs.train_fixed_parametrer_logistic_regression(target_ratio=None)
exp_fs.tune_and_train_logistic_regression(
    param_space=PARAM_SPACE, cv=2, n_initial=2, n_iterations=10, n_jobs=4, stop_iter=2,
)

Starting Initial Phase:
Using 4 parallel jobs
Initial grid:
          C  l1_ratio
0  0.003674  0.280561
1  0.713683  0.651316
Initial evaluation 1/2
Score: 0.0069 +- 0.0045 (std. err.)
------------------------------------------------------------
Initial evaluation 2/2
Score: 0.1347 +- 0.0028 (std. err.)
------------------------------------------------------------
Starting Bayesian Phase:
Bayesian iteration 1/10
Score: 0.1534 +- 0.0214 (std. err.)
New best score!
------------------------------------------------------------
Bayesian iteration 2/10
Score: 0.1527 +- 0.0276 (std. err.)
No improvement for 1 / 2 iteration(s)
------------------------------------------------------------
Bayesian iteration 3/10
Score: 0.1459 +- 0.0224 (std. err.)
No improvement for 2 / 2 iteration(s)
------------------------------------------------------------
Early stopping: No improvement for 2 iterations
Best score achieved: 0.1534
Stopped at iteration 3/10
Selecting best parameters according to 1-SE rule...
